In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from torch.utils.data import Dataset
import os
import copy
from tensorboardX import SummaryWriter
from PIL import Image
import torchvision.transforms as transforms

plt.ion()   # interactive mode
%matplotlib inline

In [2]:
class CustomDataset(Dataset):
    def __init__(self, root='D:\\Research\\data\\UCMerced_LandUse\\Images', transforms_=None, idx=[]):
        self.transform = transforms_
        self.files = []

        for path, subdirs, files in os.walk(root):
            for name in files:
                self.files.append(os.path.join(path, name))
        
        self.files =  [self.files[i] for i in idx]
        self.n = len(self.files)
        
        classes = os.listdir(root)
        self.classes = {}
        
        for i, cls in enumerate(classes):
            self.classes[cls] = i
        print(self.classes)


    def __getitem__(self, index):
        filepath = self.files[index % len(self.files)]

        cls_name = filepath.split('\\')[-2]
        img = self.transform(Image.open(filepath))
        label = self.classes[cls_name]

        return img, label

    def __len__(self):
        return len(self.files)

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

class_names = 21
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_sizes = {'val': 210, 'train': 2100-210}


In [4]:
from tqdm import tqdm

def train_model(model, criterion, optimizer, scheduler, dataloaders, writer=None, i=0,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

            writer.add_scalar(phase+"/loss", epoch_loss, epoch)
            writer.add_scalar(phase+"/acc", epoch_acc, epoch)


            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                # torch.save(model.state_dict(), 'saved/resnet_'+str(i)+'.pt')

        print()
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

# Resnet 152

In [5]:
accuracies = []
np.random.seed(0)
index = list(np.random.permutation(2100))


for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=4,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet152(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 473/473 [02:05<00:00,  3.77it/s]


train Loss: 1.6459 Acc: 0.5492


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:04<00:00, 12.72it/s]


val Loss: 0.7126 Acc: 0.8952

Epoch 1/19
----------


...

val Loss: 0.1586 Acc: 0.9762

Epoch 19/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 473/473 [02:22<00:00,  3.31it/s]


train Loss: 0.0912 Acc: 0.9799


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:04<00:00, 10.72it/s]


val Loss: 0.8307 Acc: 0.9429

Training complete in 43m 40s
Best val Acc: 0.976190



------------------------------------
 Accuracies  [0.9761904761904763]





In [6]:
torch.save(model_ft.state_dict(), 'saved/resnet_152.pt')

In [7]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}


100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:14<00:00, 14.09it/s]


[[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0],
 

In [8]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)


Row Sum: 
 [10  9 14 10 13 11  4 11  9  6 10 14  4 11  6 17  7  8  9 15 12]

Col Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  5 17  7  8  9 12 12]

Precision
 [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.75       1.         0.83333333 1.         1.         1.
 1.         0.8        1.        ]

Recall
 [1.         1.         1.         0.76923077 1.         1.
 0.8        1.         1.         1.         1.         0.93333333
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]

Model average prcision:  0.9706349206349207
Model average recall:  0.9763125763125762
Model accuracy:  0.9761904761904762


# Resnet 101


In [9]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet101(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")
    
torch.save(model_ft.state_dict(), 'saved/resnet_101.pt')

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [01:13<00:00,  3.24it/s]


train Loss: 1.6902 Acc: 0.5836


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00, 10.15it/s]


val Loss: 0.4060 Acc: 0.9048

Epoch 1/19
----------


...

val Loss: 0.3020 Acc: 0.9429

Epoch 19/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [01:13<00:00,  3.21it/s]


train Loss: 0.0679 Acc: 0.9868


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00,  9.31it/s]


val Loss: 0.2429 Acc: 0.9381

Training complete in 24m 54s
Best val Acc: 0.961905



------------------------------------
 Accuracies  [0.9761904761904763, 0.961904761904762]





In [10]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}


100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:09<00:00, 21.60it/s]


[[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0],
 

In [11]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)


Row Sum: 
 [10  9 16 12 13 11  4 11  9  5 10 13  4 12  6 17  7  8  9 12 12]

Col Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  5 17  7  8  9 12 12]

Precision
 [1.         1.         0.875      0.91666667 0.92307692 1.
 1.         1.         1.         1.         1.         1.
 0.75       0.91666667 0.83333333 1.         0.85714286 1.
 1.         1.         1.        ]

Recall
 [1.         1.         1.         0.84615385 0.92307692 1.
 0.8        1.         1.         0.83333333 1.         0.86666667
 1.         1.         1.         1.         0.85714286 1.
 1.         1.         1.        ]

Model average prcision:  0.9558041165184021
Model average recall:  0.9583987441130299
Model accuracy:  0.9619047619047619


# Resnet 50

In [5]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")
    
torch.save(model_ft.state_dict(), 'saved/resnet_50.pt')

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [02:01<00:00,  1.95it/s]


train Loss: 1.7577 Acc: 0.5614


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:08<00:00,  3.24it/s]


val Loss: 0.4534 Acc: 0.9190

Epoch 1/19
----------


...

100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [00:42<00:00,  5.52it/s]


train Loss: 0.0749 Acc: 0.9878


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 15.44it/s]


val Loss: 0.0899 Acc: 0.9810

Training complete in 16m 34s
Best val Acc: 0.990476


NameError: name 'accuracies' is not defined

In [6]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}


100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:05<00:00, 40.65it/s]


[[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0],
 

In [7]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)


Row Sum: 
 [10  9 14 13 13 11  4 11  9  6 10 14  4 11  6 17  7  8  9 12 12]

Col Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  5 17  7  8  9 12 12]

Precision
 [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.75       1.         0.83333333 1.         1.         1.
 1.         1.         1.        ]

Recall
 [1.         1.         1.         1.         1.         1.
 0.8        1.         1.         1.         1.         0.93333333
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]

Model average prcision:  0.9801587301587302
Model average recall:  0.9873015873015873
Model accuracy:  0.9904761904761905


# Resnet 34

In [8]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet34(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    print("\n\n")
    
torch.save(model_ft.state_dict(), 'saved/resnet_34.pt')

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [00:29<00:00,  7.91it/s]


train Loss: 1.5898 Acc: 0.5942


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 18.82it/s]


val Loss: 0.4304 Acc: 0.9095

Epoch 1/19
----------


...

val Loss: 0.0848 Acc: 0.9905

Epoch 19/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [00:30<00:00,  7.72it/s]


train Loss: 0.0725 Acc: 0.9873


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 17.98it/s]


val Loss: 0.0905 Acc: 0.9810

Training complete in 10m 37s
Best val Acc: 0.990476





In [9]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}


100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:04<00:00, 50.49it/s]


[[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0],
 

In [10]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)


Row Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  6 17  7  8  9 12 11]

Col Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  5 17  7  8  9 12 12]

Precision
 [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.93333333
 1.         1.         0.83333333 1.         1.         1.
 1.         1.         1.        ]

Recall
 [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.93333333
 1.         1.         1.         1.         1.         1.
 1.         1.         0.91666667]

Model average prcision:  0.9888888888888888
Model average recall:  0.9928571428571429
Model accuracy:  0.9904761904761905


# Resnet 18

In [5]:
np.random.seed(0)
index = list(np.random.permutation(2100))

for i in range(1):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
torch.save(model_ft.state_dict(), 'saved/resnet_18.pt')

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [00:19<00:00, 12.05it/s]


train Loss: 1.7059 Acc: 0.5550


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 26.72it/s]


val Loss: 0.4864 Acc: 0.8714

Epoch 1/19
----------


...

val Loss: 0.0746 Acc: 0.9905

Epoch 19/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [00:19<00:00, 12.03it/s]


train Loss: 0.0966 Acc: 0.9820


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 23.48it/s]


val Loss: 0.0723 Acc: 0.9905

Training complete in 6m 35s
Best val Acc: 0.995238


In [6]:
model = model_ft
model.eval()
result = [[0 for _ in range(21)] for _ in range(21)]

dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=1,
                                             shuffle=True)
              for x in ['train', 'val']}

for inputs, labels in tqdm(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    
    row = preds.data[0].cpu().numpy()
    col = labels.data[0].cpu().numpy() 
    result[row][col]+=1

result

{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
{'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}


100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:02<00:00, 91.03it/s]


[[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0],
 

In [7]:
precision = np.zeros(21)
recall = np.zeros(21)

npr = np.array(result)
row_sum = npr.sum(axis=1)
print("\nRow Sum: \n", row_sum)
col_sum = npr.sum(axis=0)
print("\nCol Sum: \n", col_sum)
total_correct = 0

for i in range(21):
    precision[i] = result[i][i] / row_sum[i]
    recall[i] = result[i][i] / col_sum[i]
    total_correct += result[i][i]

print("\nPrecision\n", precision)
print("\nRecall\n", recall)


print()
print("Model average prcision: ", np.mean(precision))
print("Model average recall: ", np.mean(recall))
print("Model accuracy: ", total_correct/210)


Row Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 14  3 11  6 17  7  8  9 12 12]

Col Sum: 
 [10  9 14 13 13 11  5 11  9  6 10 15  3 11  5 17  7  8  9 12 12]

Precision
 [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.83333333 1.         1.         1.
 1.         1.         1.        ]

Recall
 [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.93333333
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]

Model average prcision:  0.9920634920634922
Model average recall:  0.9968253968253968
Model accuracy:  0.9952380952380953
